# Scrambling of operators

In [1]:
using ITensors
using ITensorTDVP
using Plots
using Observers
using LinearAlgebra

ArgumentError: ArgumentError: Package ITensors [9136182c-28ba-11e9-034c-db9fb085ebd5] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.


In [2]:
N  = 21     # Number of spins
J  = 1.0    # ZZ interaction strength
hx = 1.05   # X-field 
hz = 0.5    # Z-field
δt = 0.05   # Time-step for evolution
T  = 5.0    # Total time
χ  = 32;    # Max link dimension allowed

## System

In [3]:
sites = siteinds("S=1/2",N);      # Make N S=1/2 spin indices defining system

UndefVarError: UndefVarError: `siteinds` not defined

In [4]:
# Define mixed-field Ising Hamiltionian operator terms:

H_op = OpSum()
for i=1:N-1
    H_op += 1.0,"Sz",i,"Sz",i+1 # ZZ terms
end
for i=1:N
    H_op +=  hx,"Sx",i         # X terms
    H_op +=  hz,"Sz",i         # Z terms
end
# Convert these terms to an MPO
H = MPO(H_op,sites);

UndefVarError: UndefVarError: `OpSum` not defined

In [5]:
# Perform DMRG to find the ground state:

ψ0 = randomMPS(sites,2)               # Random initial state
sweeps = Sweeps(10)             # Number of sweeps is 10  
# sweeps is some form of iterator/object with info about cutoff, maxdim, mindim, noise
# it will be 10 long
maxdim!(sweeps,10,20,100,100,200) # Gradually increase states kept
# changes the maxdim aspect of sweeps to be 10,20,100,100,200,200,200,200,200,200 - fills in last value until end
# ! means that input argument is altered - by convention
cutoff!(sweeps,1E-10)                 # Desired truncation error
# repeats with cutoff
en_min,ψ_gs = dmrg(H,ψ0,sweeps);      # Find the ground state energy 
# does dmrg using this sweeps object

UndefVarError: UndefVarError: `randomMPS` not defined

In [6]:
zzcorr = correlation_matrix(ψ_gs,"Sz","Sz");
gr()
heatmap(1:N, 1:N, zzcorr, c = :heat)

UndefVarError: UndefVarError: `correlation_matrix` not defined

In [7]:
zzcen = diag(reverse(zzcorr, dims = 2));
plot(1:N, zzcen, label=false)  
scatter!(1:N, zzcen, label=false) 

UndefVarError: UndefVarError: `zzcorr` not defined

## Extended system + ancilla

In [8]:
sitesext = siteinds("S=1/2",2*N); # Make 2N S=1/2 spin indices defining system + ancilla

UndefVarError: UndefVarError: `siteinds` not defined

In [9]:
# Extract the raising, lowering and identity operators for the extended system:

Sp = ops(sitesext, [("S+", n) for n in 1:(2*N)]);  # Raising operators
Sm = ops(sitesext, [("S-", n) for n in 1:(2*N)]);  # Lowering operators
# ITensors doesn't include the identity operator as standard so construct it:
Id = Vector{ITensor}(undef,2*N)
for i =1:(2*N)
    iv = sitesext[i]
    ID = ITensor(iv', dag(iv));
    for j in 1:ITensors.dim(iv)
        ID[iv' => j, iv => j] = 1.0
    end
    Id[i] = ID
end

UndefVarError: UndefVarError: `sitesext` not defined

In [10]:
# Construct the identity vacuum state:

Ivac = MPS(sitesext, "Up") # All up spins initial state
gates = [(Id[n]*Id[n+1] + Sm[n]*Sm[n+1]) for n in 1:2:(2*N)]; # Maps |00> => |00> + |11>
Ivac = apply(gates, Ivac; cutoff=1e-15); # Note we have no 1/sqrt(2) normalisation

UndefVarError: UndefVarError: `MPS` not defined

In [11]:
# Vectorisation approach used here is to stack matrix rows into a column vector.
# This means that:
# vec(AB) = A ⊗ I vec(B) =  I ⊗ B^T vec(A)
# so |i><j| => |i> ⊗ |j>
# vec(L A R) = L ⊗ R^T vec(A)

# Define "Commutator" Hamiltonian operator terms:

# HC = H ⊗ I - I ⊗ H, since H is real and hermitian H = H^T.
H_op = OpSum()
for i=1:2*(N-1)
    H_op += (-1)^(i-1) *  J,"Sz",i,"Sz",i+2
end
for i=1:2*N
    H_op += (-1)^(i-1) *  hx,"Sx",i
    H_op += (-1)^(i-1) *  hz,"Sz",i
end
# Convert these terms to an MPO
HC = MPO(H_op,sitesext);

UndefVarError: UndefVarError: `OpSum` not defined

In [12]:
# Define observable for scrambling:

A_op = OpSum()
A_op += 1.0,"Sx",2*floor(Int,N/2+1)-1  # Sx operator in the middle of the system
A = MPO(A_op,sitesext);                # Build the MPO from these terms
Avec = apply(A, Ivac; cutoff=1e-15);   # Compute |A> = A|I>

UndefVarError: UndefVarError: `OpSum` not defined

## Perform TDVP time evolution of the operator

In [13]:
# Define function for computing entanglement entropy

function entanglement_entropy(ψ)
# Compute the von Neumann entanglement entropy across each bond of the MPS
    N = length(ψ)
    SvN = zeros(N)
    psi = ψ
    for b=1:N
        psi = orthogonalize(psi, b)
        if b==1
            U,S,V = svd(psi[b] , siteind(psi, b))
        else
            U,S,V = svd(psi[b], (linkind(psi, b-1), siteind(psi, b)))
        end
        p = diag(S).^2               # Extract square of Schmidt coefficients
        p = p ./ sum(p)              # Normalise to a probability dist
        SvN[b] = -sum(p .* log2.(p)) # Compute Shannon entropy
    end
    return SvN
end;

In [14]:
SvN_init = entanglement_entropy(Avec);
plot(1:(2*N), SvN, label=false) 

UndefVarError: UndefVarError: `Avec` not defined

Here $|A>$ displays the entanglement structure of the initial vacuum state.

In [15]:
# Define observer functions for TDVP:

function current_time(; current_time, bond, half_sweep)
  if bond == 1 && half_sweep == 2
    return real(-im*current_time)
  end
    
  return nothing
end

function measure_SvN(; psi, bond, half_sweep)
  if bond == 1 && half_sweep == 2
    return entanglement_entropy(psi)-SvN_init
  end
  return nothing
end;

function measure_linkdim(; psi, bond, half_sweep)
  if bond == 1 && half_sweep == 2
    return maxlinkdim(psi)
  end
  return nothing
end;

In [16]:
# Perform TDVP evolution of |A(t)>:

obs = Observer("times" => current_time, "SvN" => measure_SvN, "chi" => measure_linkdim)

# d|A(t)>/dt = i HC |A(t)> so |A(t)> = exp(i t HC)|A(0)> 

ψf = tdvp(HC, im * T, Avec; 
          time_step = im * δt,
          normalize = false, 
          maxdim = χ,
          cutoff = 1e-10,
          outputlevel=1,
          (observer!)=obs)

# Extract results from time-step observations
#`observer.name` or `observer[!, "name"]`
#res = results(obs)
times = obs.times
SvN = obs.SvN
chi = obs.chi;
# times = res["times"]
# SvN = res["SvN"]
# chi = res["chi"];

UndefVarError: UndefVarError: `Observer` not defined

In [17]:
# Plot the entanglement entropy of each bond for system + ancilla:
gr()
heatmap(1:(2*N), times, reduce(vcat,transpose.(SvN)), c = :heat)

UndefVarError: UndefVarError: `gr` not defined

In [18]:
# Plot the entanglement entropy for bonds separating system + ancilla pairs:
gr()
S = reduce(vcat,transpose.(SvN))[:,2:2:(2*N)]
heatmap(1:N, times, S, c = :heat)

UndefVarError: UndefVarError: `gr` not defined

In [19]:
# Plot entanglement entropy of bonds between system + ancilla pairs:
gr()
S = reduce(vcat,transpose.(SvN))[:,1:2:(2*N)]
heatmap(1:N, times, S, c = :heat)

UndefVarError: UndefVarError: `gr` not defined

In [20]:
# Plot the growth in the maximum link dimension with time:
plot(times, chi, label=false)  
scatter!(times, chi, label=false) 

UndefVarError: UndefVarError: `plot` not defined